This code read and calculate weights on the rainfall dataset. I realized the errors in my previous code later, so the previous code may not work now...

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
data_yrs = ['0609', '0911', '1113']
temp_path = rf'G:\Shared drives\Ryoko and Hilary\SMSigxModel\analysis\0_data\Little Washita\data_ars_temp'
out_path = r'G:\Shared drives\Ryoko and Hilary\SMSigxModel\analysis\2_data_input\LittleWashita'


## (1) First, concatinate all data within a folder and store them in the temporary folder

In [ ]:
for n_yr in range(len(data_yrs)):
    # Extract the dates and sensor numbers from the file names
    folder_path = rf'G:\Shared drives\Ryoko and Hilary\SMSigxModel\analysis\0_data\Little Washita\data_ars_{data_yrs[n_yr]}'
    # Get a list of all files in the folder
    file_list = os.listdir(folder_path)
    # file_list[1].split('a')[1][0:3]

    dates = [file.split('a')[0] for file in file_list if file != 'README']
    sensors = [file.split('a')[1][0:3] for file in file_list if file != 'README']
    
    # %%
    unique_dates = list(set(dates))
    unique_sensors = list(set(sensors))
    # %%
    # Create lists to store the dataframes for each sensor
    sensor_dataframes = {}

    # for sensor in unique_sensors:
    #     # Create an empty dataframe for the sensor
    #     sensor_dataframes[sensor] = pd.DataFrame()
    for i, sensor in enumerate(unique_sensors):
        sensor_dataframes[sensor] = pd.DataFrame()
        print(f'Currently processing sensor {i+1}/{len(unique_sensors)}')
        
        for date in tqdm(unique_dates):
            # Extract the date and sensor number from the file name
            file_name = os.path.join(folder_path, date + 'a' + sensor + '.txt')
            
            # Read the file as a pandas dataframe
            file_path = os.path.join(folder_path, file_name)
            
            if os.path.exists(file_path):
                df = pd.read_csv(file_path, delim_whitespace=True, header=3)
                
                df['date'] = pd.to_datetime(date, format='%Y%m%d')
                
                mask = df['HR'] == 24
                df.loc[mask, 'date'] = df.loc[mask, 'date'] + pd.Timedelta(days=1)
                df.loc[mask, 'HR'] = '00'
                
                df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['HR'].astype(str).str.zfill(2) + ':'+ df['MN'].astype(str).str.zfill(2), format='%Y-%m-%d %H:%M')
                
                df.loc[df["QRN"] != "g", "RAIN"] = np.nan
                df["RAIN"] = df["RAIN"].diff() # Convert cumulative to original

                df = df[["datetime", "RAIN"]]

                # Concatenate the dataframe into the dataframe for the sensor
                sensor_dataframes[sensor] = pd.concat([sensor_dataframes[sensor], df], axis=0, ignore_index=True)
            else:
                pass
        
        # Save the dataframe for the sensor to a CSV file
        file_name = 'ars' + data_yrs[n_yr] + '_a' + sensor + '_rain.csv'
        file_path = os.path.join(temp_path, file_name)
        sensor_dataframes[sensor].to_csv(file_path, index=False)

# (2) Then, concatinete sensors for the all period. Drop nan values, make time interval regular. 

In [ ]:

temp_path_2 = rf'G:\Shared drives\Ryoko and Hilary\SMSigxModel\analysis\0_data\Little Washita\data_ars_temp2'
temp_path = rf'G:\Shared drives\Ryoko and Hilary\SMSigxModel\analysis\0_data\Little Washita\data_ars_temp'
# # Get the ones that has record from 2006
# sensors = [file.split('0609_a')[1][0:3] for file in file_list if (file != 'README') and ('0609' in file)]
n_yr = 0
file_list = os.listdir(temp_path)
print(file_list)
sensors = [file.split('_a')[1][0:3] for file in file_list if (file != 'README') and ('0609' in file)]
unique_sensors = list(set(sensors))
print(unique_sensors)
len(unique_sensors)